In [1]:
# PS3 - CE264
# GSI: Mustapha Harb - Mengqiao Yu - Andrew Campbell

# importing the requried libraries
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format

# reading the data file 
wide_swiss_metro  = pd.read_csv("swissmetro.csv",sep=",")

In [2]:
# Create the list of individual specific variables
ind_variables = wide_swiss_metro.columns.tolist()[:15]

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'TRAIN_TT'),
                                               (2, 'SM_TT'),
                                               (3, 'CAR_TT')]),
                          u'travel_cost': dict([(1, 'TRAIN_CO'),
                                                (2, 'SM_CO'),
                                                (3, 'CAR_CO')]),
                          u'headway': dict([(1, 'TRAIN_FR'),
                                            (2, 'SM_FR')]),
                          u'seat_configuration': dict([(2, "SM_SEATS")])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'TRAIN_AV',
                          2: 'SM_AV', 
                          3: 'CAR_AV'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
wide_swiss_metro[obs_id_column] = np.arange(wide_swiss_metro.shape[0],
                                            dtype=int) + 1


# Create a variable recording the choice column
choice_column = "CHOICE"

In [3]:
# Perform the conversion to long-format
long_swiss_metro = pl.convert_wide_to_long(wide_swiss_metro, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_swiss_metro.head(10).T

,0,1,2,3,4,5,6,7,8,9
custom_id,1.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0
mode_id,1.0,2.0,3.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0
CHOICE,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
GROUP,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
SURVEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ID,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
PURPOSE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
FIRST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TICKET,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
##########
# Create scaled variables so the estimated coefficients are of similar magnitudes
##########
# Scale the travel time column by 60 to convert raw units (minutes) to hours
long_swiss_metro["travel_time_hrs"] = long_swiss_metro["travel_time"] / 60.0

# Scale the headway column by 60 to convert raw units (minutes) to hours
long_swiss_metro["headway_hrs"] = long_swiss_metro["headway"] / 60.0

# specifying the utility equations

# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']

basic_specification["travel_time_hrs"] = [[1, 2,], 3]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (Train and Swissmetro)',
                                  'Travel Time, units:hrs (Car)']

basic_specification["headway_hrs"] = [1, 2]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train)",
                              "Headway, units:hrs, (Swissmetro)"]

In [5]:
# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_swiss_metro,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(6))



Log-likelihood at zero: -11,093.6273
Initial Log-likelihood: -11,093.6273


/home/Carlin/anaconda3/envs/python2/lib/python2.7/site-packages/scipy/optimize/_minimize.py:420: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.71 seconds.
Final log-likelihood: -8,849.6608


In [6]:
# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:               10,719
Model:             Multinomial Logit Model   Df Residuals:                   10,713
Method:                                MLE   Df Model:                            6
Date:                     Wed, 21 Feb 2018   Pseudo R-squ.:                   0.202
Time:                             22:38:56   Pseudo R-bar-squ.:               0.202
AIC:                            17,711.322   Log-Likelihood:             -8,849.661
BIC:                            17,755.000   LL-Null:                   -11,093.627
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
ASC Train                                         0.4385      0.092      4.792      0.000       0.259       0.618
ASC Swissmetro                                    0.5760      0.077      7.471      0.000       0.425       0.727
Travel Time, units:hrs (Train and Swissmetro)    -0.9125      0.030    -30.048      0.000      -0.972      -0.853
Travel Time, units:hrs (Car)                     -0.5915      0.026    -22.483      0.000      -0.643      -0.540
Headway, units:hrs, (Train)                      -0.3502      0.049     -7.179      0.000      -0.446      -0.255
Headway, units:hrs, (Swissmetro)                 -0.3940      0.152     -2.592      0.010      -0.692      -0.096
=================================================================================================================
"""

# Model Creation

## Modifying for GA

First we have to modify the cost variable so that everyone who has a season ticket sees their cost utility go to zero, as different interpretations of the questions skewed their data in a way that would muddle our results.

In [18]:
long_swiss_metro['revised_travel_cost'] = (long_swiss_metro['GA']==0)*long_swiss_metro['travel_cost']
(long_swiss_metro['revised_travel_cost'] == 0).sum()

3771

## $\alpha x$ values

Next let's create the rest of our $\alpha\beta$ coeffients, wrapping up the creation of the $\alpha$ terms and mixing them with the $x$ variables in one step.

In [31]:
# traveling first classs effect on cost sensitivy
long_swiss_metro['1c'] = long_swiss_metro['revised_travel_cost'] * long_swiss_metro['FIRST']
(long_swiss_metro['1c']>0).sum()

13428

In [32]:
# using free-after-7-pm tickets effect on cost sensitivity
long_swiss_metro['ft'] = long_swiss_metro['revised_travel_cost'] * (long_swiss_metro['TICKET'] == 8)
(long_swiss_metro['ft']>0).sum() # only a small percentage of the dataset wre using these free-after-7-pm tickets

756

In [33]:
# self-paying effect on cost sensitivity 
long_swiss_metro['sp'] = long_swiss_metro['revised_travel_cost'] * long_swiss_metro['WHO'].isin([1,3])
(long_swiss_metro['sp']>0).sum()

16218

In [30]:
# traveling for business effect on travel time sensitivy
long_swiss_metro['bp'] = long_swiss_metro['travel_time_hrs'] * long_swiss_metro['PURPOSE'].isin([1, 3, 5, 7])
(long_swiss_metro['bp']>0).sum()

19944

## Building and testing our specs

With our variables created, next we lay out the specs for our model and test.

In [35]:
basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']


# cost variables

basic_specification["revised_travel_cost"] = [[1, 2, 3]]
basic_names["revised_travel_cost"] = ["Cost Revised for GA, units:CHF, (All choices)"]

basic_specification["1c"] = [[1, 2, 3]]
basic_names["1c"] = ["Cost Revised x First Class, units:CHF, (All choices)"]

basic_specification["ft"] = [[1, 2, 3]]
basic_names["ft"] = ["Cost Revised x Using 7pm-free Tickets, units:CHF, (All choices)"]

basic_specification["sp"] = [[1, 2, 3]]
basic_names["sp"] = ["Cost Revised x Self-Pay, units:CHF, (All choices)"]



# travel time coefficients

basic_specification["travel_time_hrs"] = [[1, 2, 3]]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (All choices)']

basic_specification["bp"] = [[1, 2, 3]]
basic_names["bp"] = ['Travel Time x Traveling for Business, units:hrs (All choices)']



# headway coefficient
basic_specification["headway_hrs"] = [[1, 2]]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train and Swissmetro)"]



# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_swiss_metro,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(9))

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -11,093.6273
Initial Log-likelihood: -11,093.6273
Estimation Time for Point Estimation: 1.21 seconds.
Final log-likelihood: -8,481.0506


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:               10,719
Model:             Multinomial Logit Model   Df Residuals:                   10,710
Method:                                MLE   Df Model:                            9
Date:                     Thu, 22 Feb 2018   Pseudo R-squ.:                   0.236
Time:                             00:02:21   Pseudo R-bar-squ.:               0.235
AIC:                            16,980.101   Log-Likelihood:             -8,481.051
BIC:                            17,045.619   LL-Null:                   -11,093.627
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
ASC Train                                                          -0.2233      0.062     -3.605      0.000      -0.345      -0.102
ASC Swissmetro                                                      0.1188      0.036      3.292      0.001       0.048       0.189
Cost Revised for GA, units:CHF, (All choices)                      -0.0173      0.001    -15.996      0.000      -0.019      -0.015
Cost Revised x First Class, units:CHF, (All choices)                0.0059      0.001      5.816      0.000       0.004       0.008
Cost Revised x Using 7pm-free Tickets, units:CHF, (All choices)     0.0188      0.001     17.473      0.000       0.017       0.021
Cost Revised x Self-Pay, units:CHF, (All choices)                1.243e-05      0.001      0.016      0.987      -0.001       0.002
Travel Time, units:hrs (All choices)                               -0.7896      0.035    -22.418      0.000      -0.859      -0.721
Travel Time x Traveling for Business, units:hrs (All choices)      -0.0775      0.037     -2.102      0.036      -0.150      -0.005
Headway, units:hrs, (Train and Swissmetro)                         -0.3642      0.046     -7.836      0.000      -0.455      -0.273
===================================================================================================================================
"""

This looks better than the more naive model enumerated above, but let's try making the cost coefficients alternative-specific between transit (train/Swissmetro) and car.

In [36]:
basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']


# cost variables

basic_specification["revised_travel_cost"] = [[1, 2], 3]
basic_names["revised_travel_cost"] = ["Cost Revised for GA, units:CHF, (Train and Swissmetro)",
                                      "Cost Revised for GA, units:CHF, (Car)"]

basic_specification["1c"] = [[1, 2], 3]
basic_names["1c"] = ["Cost Revised x First Class, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x First Class, units:CHF, (Car)"]

basic_specification["ft"] = [[1, 2], 3]
basic_names["ft"] = ["Cost Revised x Using 7pm-free Tickets, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x Using 7pm-free Tickets, units:CHF, (Car)"]

basic_specification["sp"] = [[1, 2], 3]
basic_names["sp"] = ["Cost Revised x Self-Pay, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x Self-Pay, units:CHF, (Car)"]



# travel time coefficients

basic_specification["travel_time_hrs"] = [[1, 2, 3]]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (All choices)']

basic_specification["bp"] = [[1, 2, 3]]
basic_names["bp"] = ['Travel Time x Traveling for Business, units:hrs (All choices)']



# headway coefficient
basic_specification["headway_hrs"] = [[1, 2]]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train and Swissmetro)"]



# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_swiss_metro,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(13))

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -11,093.6273
Initial Log-likelihood: -11,093.6273
Estimation Time for Point Estimation: 1.55 seconds.
Final log-likelihood: -8,276.2939


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:               10,719
Model:             Multinomial Logit Model   Df Residuals:                   10,706
Method:                                MLE   Df Model:                           13
Date:                     Thu, 22 Feb 2018   Pseudo R-squ.:                   0.254
Time:                             00:08:40   Pseudo R-bar-squ.:               0.253
AIC:                            16,578.588   Log-Likelihood:             -8,276.294
BIC:                            16,673.225   LL-Null:                   -11,093.627
============================================================================================================================================
                                                                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------
ASC Train                                                                    0.6085      0.080      7.569      0.000       0.451       0.766
ASC Swissmetro                                                               0.8861      0.057     15.481      0.000       0.774       0.998
Cost Revised for GA, units:CHF, (Train and Swissmetro)                      -0.0222      0.001    -19.309      0.000      -0.024      -0.020
Cost Revised for GA, units:CHF, (Car)                                       -0.0094      0.001     -7.160      0.000      -0.012      -0.007
Cost Revised x First Class, units:CHF, (Train and Swissmetro)                0.0086      0.001      8.108      0.000       0.007       0.011
Cost Revised x First Class, units:CHF, (Car)                                 0.0023      0.001      1.906      0.057    -6.6e-05       0.005
Cost Revised x Using 7pm-free Tickets, units:CHF, (Train and Swissmetro)     0.0174      0.002     11.225      0.000       0.014       0.020
Cost Revised x Using 7pm-free Tickets, units:CHF, (Car)                     -0.0242      0.011     -2.150      0.032      -0.046      -0.002
Cost Revised x Self-Pay, units:CHF, (Train and Swissmetro)                  -0.0001      0.001     -0.129      0.897      -0.002       0.001
Cost Revised x Self-Pay, units:CHF, (Car)                                    0.0012      0.001      1.120      0.263      -0.001       0.003
Travel Time, units:hrs (All choices)                                        -0.9659      0.038    -25.554      0.000      -1.040      -0.892
Travel Time x Traveling for Business, units:hrs (All choices)               -0.0111      0.039     -0.287      0.774      -0.087       0.065
Headway, units:hrs, (Train and Swissmetro)                                  -0.3572      0.047     -7.641      0.000      -0.449      -0.266
============================================================================================================================================
"""

Let's also try making the travel time coefficients alternative-specific between transit (train/Swissmetro) and car instead.

In [37]:
basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']


# cost variables

basic_specification["revised_travel_cost"] = [[1, 2, 3]]
basic_names["revised_travel_cost"] = ["Cost Revised for GA, units:CHF, (All choices)"]

basic_specification["1c"] = [[1, 2, 3]]
basic_names["1c"] = ["Cost Revised x First Class, units:CHF, (All choices)"]

basic_specification["ft"] = [[1, 2, 3]]
basic_names["ft"] = ["Cost Revised x Using 7pm-free Tickets, units:CHF, (All choices)"]

basic_specification["sp"] = [[1, 2, 3]]
basic_names["sp"] = ["Cost Revised x Self-Pay, units:CHF, (All choices)"]



# travel time coefficients

basic_specification["travel_time_hrs"] = [[1, 2], 3]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (Train and Swissmetro)',
                                  "Travel Time, units:hrs (Car)"]

basic_specification["bp"] = [[1, 2], 3]
basic_names["bp"] = ['Travel Time x Traveling for Business, units:hrs (Train and Swissmetro)',
                     "Travel Time x Traveling for Business, units:hrs (Car)"]



# headway coefficient
basic_specification["headway_hrs"] = [[1, 2]]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train and Swissmetro)"]



# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_swiss_metro,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(11))

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -11,093.6273
Initial Log-likelihood: -11,093.6273
Estimation Time for Point Estimation: 1.79 seconds.
Final log-likelihood: -8,361.6369


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:               10,719
Model:             Multinomial Logit Model   Df Residuals:                   10,708
Method:                                MLE   Df Model:                           11
Date:                     Thu, 22 Feb 2018   Pseudo R-squ.:                   0.246
Time:                             00:11:36   Pseudo R-bar-squ.:               0.245
AIC:                            16,745.274   Log-Likelihood:             -8,361.637
BIC:                            16,825.351   LL-Null:                   -11,093.627
==========================================================================================================================================
                                                                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
ASC Train                                                                  0.4478      0.094      4.747      0.000       0.263       0.633
ASC Swissmetro                                                             0.6652      0.065     10.284      0.000       0.538       0.792
Cost Revised for GA, units:CHF, (All choices)                             -0.0190      0.001    -17.166      0.000      -0.021      -0.017
Cost Revised x First Class, units:CHF, (All choices)                       0.0072      0.001      6.950      0.000       0.005       0.009
Cost Revised x Using 7pm-free Tickets, units:CHF, (All choices)            0.0191      0.001     17.202      0.000       0.017       0.021
Cost Revised x Self-Pay, units:CHF, (All choices)                          0.0006      0.001      0.799      0.424      -0.001       0.002
Travel Time, units:hrs (Train and Swissmetro)                             -1.0986      0.044    -25.128      0.000      -1.184      -1.013
Travel Time, units:hrs (Car)                                              -0.6455      0.037    -17.466      0.000      -0.718      -0.573
Travel Time x Traveling for Business, units:hrs (Train and Swissmetro)     0.1646      0.045      3.681      0.000       0.077       0.252
Travel Time x Traveling for Business, units:hrs (Car)                     -0.0974      0.038     -2.568      0.010      -0.172      -0.023
Headway, units:hrs, (Train and Swissmetro)                                -0.3592      0.047     -7.693      0.000      -0.451      -0.268
==========================================================================================================================================
"""

And what if both cost and travel time were alternative-specific?

In [38]:
basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']


# cost variables

basic_specification["revised_travel_cost"] = [[1, 2], 3]
basic_names["revised_travel_cost"] = ["Cost Revised for GA, units:CHF, (Train and Swissmetro)",
                                      "Cost Revised for GA, units:CHF, (Car)"]

basic_specification["1c"] = [[1, 2], 3]
basic_names["1c"] = ["Cost Revised x First Class, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x First Class, units:CHF, (Car)"]

basic_specification["ft"] = [[1, 2], 3]
basic_names["ft"] = ["Cost Revised x Using 7pm-free Tickets, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x Using 7pm-free Tickets, units:CHF, (Car)"]

basic_specification["sp"] = [[1, 2], 3]
basic_names["sp"] = ["Cost Revised x Self-Pay, units:CHF, (Train and Swissmetro)",
                     "Cost Revised x Self-Pay, units:CHF, (Car)"]



# travel time coefficients

basic_specification["travel_time_hrs"] = [[1, 2], 3]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (Train and Swissmetro)',
                                  "Travel Time, units:hrs (Car)"]

basic_specification["bp"] = [[1, 2], 3]
basic_names["bp"] = ['Travel Time x Traveling for Business, units:hrs (Train and Swissmetro)',
                     "Travel Time x Traveling for Business, units:hrs (Car)"]



# headway coefficient
basic_specification["headway_hrs"] = [[1, 2]]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train and Swissmetro)"]



# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_swiss_metro,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(15))

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -11,093.6273
Initial Log-likelihood: -11,093.6273
Estimation Time for Point Estimation: 1.01 seconds.
Final log-likelihood: -8,242.1188


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:               10,719
Model:             Multinomial Logit Model   Df Residuals:                   10,704
Method:                                MLE   Df Model:                           15
Date:                     Thu, 22 Feb 2018   Pseudo R-squ.:                   0.257
Time:                             00:13:44   Pseudo R-bar-squ.:               0.256
AIC:                            16,514.238   Log-Likelihood:             -8,242.119
BIC:                            16,623.434   LL-Null:                   -11,093.627
============================================================================================================================================
                                                                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------
ASC Train                                                                    0.4798      0.096      4.978      0.000       0.291       0.669
ASC Swissmetro                                                               0.8035      0.067     11.934      0.000       0.672       0.935
Cost Revised for GA, units:CHF, (Train and Swissmetro)                      -0.0222      0.001    -19.052      0.000      -0.024      -0.020
Cost Revised for GA, units:CHF, (Car)                                       -0.0082      0.001     -5.919      0.000      -0.011      -0.006
Cost Revised x First Class, units:CHF, (Train and Swissmetro)                0.0087      0.001      8.082      0.000       0.007       0.011
Cost Revised x First Class, units:CHF, (Car)                                 0.0032      0.001      2.585      0.010       0.001       0.006
Cost Revised x Using 7pm-free Tickets, units:CHF, (Train and Swissmetro)     0.0174      0.002     11.028      0.000       0.014       0.020
Cost Revised x Using 7pm-free Tickets, units:CHF, (Car)                     -0.0272      0.012     -2.313      0.021      -0.050      -0.004
Cost Revised x Self-Pay, units:CHF, (Train and Swissmetro)                  -0.0003      0.001     -0.343      0.731      -0.002       0.001
Cost Revised x Self-Pay, units:CHF, (Car)                                   -0.0013      0.001     -1.151      0.250      -0.004       0.001
Travel Time, units:hrs (Train and Swissmetro)                               -1.0444      0.044    -23.730      0.000      -1.131      -0.958
Travel Time, units:hrs (Car)                                                -0.9525      0.045    -21.054      0.000      -1.041      -0.864
Travel Time x Traveling for Business, units:hrs (Train and Swissmetro)       0.1566      0.045      3.496      0.000       0.069       0.244
Travel Time x Traveling for Business, units:hrs (Car)                       -0.0795      0.040     -1.974      0.048      -0.158      -0.001
Headway, units:hrs, (Train and Swissmetro)                                  -0.3547      0.047     -7.596      0.000      -0.446      -0.263
============================================================================================================================================
"""